In [5]:
# Importing the Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [7]:
# Importing the training dataset
comments_df = pd.read_csv("D:/Projects/toxic-comment-classification/data/cleaned-data/train-cleaned&lemmatized.csv")
comments_df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,1,000103f0d9cfb60f,daww match background colour seemingly stuck t...,0,0,0,0,0,0
2,2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,3,0001b41b1c6bb37e,make real suggestion improvement wondered sect...,0,0,0,0,0,0
4,4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0
